In [ ]:
import IBSEN
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import csv
import numpy as np
import os
import math
import matplotlib.style as mplstyle
from datetime import datetime
import time

In [ ]:
global numberOfSpectrometers
global spectrometer_data
global idx_NIR
global idx_VIS
global idx_concatenated_data
global params
global norm_factor

In [ ]:
def create_wavelengthaxis_and_dark_spectrum(exp_time):
    handle = IBSEN.InitSpectrometer(None, 0)
    dark_spectrum = IBSEN.acquireSpectra(handle, 0, exp_time, 1, -1, 1, -1, -1, 0, '', 0) 
    dark_spectrum = np.array(dark_spectrum).astype(np.float64)
    detector_length = IBSEN.get_numPixImage(handle)
    wavelengthaxis = IBSEN.createWavelengthAxis(handle, 0, detector_length-1)
    wavelengthaxis = np.array(wavelengthaxis).astype(np.float64)
    detector_type = IBSEN.get_detectorType(handle)
    IBSEN.closeDevice(handle)
    return wavelengthaxis, dark_spectrum, detector_type

In [ ]:
def create_total_wavelengthaxis(spectrometer1, spectrometer2, cutoff):
    wavelengthaxis1 = spectrometer_data[spectrometer1]['wavelengthaxis']
    wavelengthaxis1_cutted = wavelengthaxis1[wavelengthaxis1 >= cutoff]
    wavelengthaxis2 = spectrometer_data[spectrometer2]['wavelengthaxis']
    wavelengthaxis2_cutted = wavelengthaxis2[wavelengthaxis2 <= cutoff]
    wavelengthaxis_total = np.concatenate((wavelengthaxis1_cutted, wavelengthaxis2_cutted), axis=0)
    return wavelengthaxis_total

In [ ]:
def create_handle(number, serialnumber):
    handle = IBSEN.InitSpectrometer(serialnumber, number)
    return handle    

In [ ]:
#get number of connected spectrometer
numberOfSpectrometers = IBSEN.GetNumSpectrometers()
print("Es wurden {numberSpectrometers} Spektrometer detektiert.".format(numberSpectrometers = numberOfSpectrometers))

In [ ]:
if(numberOfSpectrometers > 0 ):
    #Determination of the available serial numbers
    pcb, serial = IBSEN.GetSerialNumbers(numberOfSpectrometers)
    #print the serial numbers
    for i in range(numberOfSpectrometers):
        print("Spektrometer {i} hat PCB Nummer {pcb} und Seriennummer {serie}.".format(i= i, pcb=pcb[i], serie=serial[i]))

In [ ]:
#load parameter
params = {}
with open("parameter_files/parameterslive.csv") as file:
    reader = csv.reader(file, delimiter = ',')
    for line in reader:
        params[line[0]] = line[1]
print(params)
        
#small relabelling for IBSEN
if int(params['timeScale']) == 1:
    params['timeScale'] = 'MILLI'
elif int(params['timeScale']) == 2:
    params['timeScale'] = 'MICRO'
print('Paramterfile loaded')
params

In [ ]:
##### Initialize a dict that has the serialnumbers as keys and a dict with the wavelengthaxis and dark_spectrum
#of the PEBBLE as values of the keys
# e.g. example_dict = {serialnumber : { 'wavelengthaxis' : [ 1, 2, 3, 4, 5, ,6], 'DarkSpec' : spectrum}}
# use this dict to add dark_spectrum and spectrum later on
#OBS! the function create_wavelengthaxis closes every handle that is given to the fucntion in order to work correct.
#You will need to open the handle again afterwards
spectrometer_data = {}

for i, serialnumber in enumerate(serial):
    wavelengthaxis, dark_spectrum, detector_type = create_wavelengthaxis_and_dark_spectrum(int(params['expTime']))
    spectrometer_data[i] = {'wavelengthaxis': wavelengthaxis, 'darkspec': dark_spectrum, 'detectortype': detector_type,
                            'serialnumber': serialnumber,}

#Get the detector type. Valid values are:
# 0: Hamamatsu S11639-01, S13496 or S14379 
# (ISB1)
# 1: Hamamatsu S11156 (ISB3)
# 2: Hamamatsu G11476 (ISB4)
# 3: Hamamatsu S10420 (ISB5)
# 4: Hamamatsu G13913 (ISB6
# 2 and 4 NIR 0, 1, 3 VIS

for item in spectrometer_data:
    if int(spectrometer_data[item]['detectortype']) == 2 or int(spectrometer_data[item]['detectortype']) == 4:
        spectrometer_data[item]['detectortype'] = 'NIR'
    else:
        spectrometer_data[item]['detectortype'] = 'VIS'
print("Created Wavelengthaxis, Type, DarkCounts and Handle for every connected Spectrometer ")

In [ ]:
#get keys of NIR and VIS PEBBLES to adress them later
idx_NIR = []
idx_VIS = []
for spectrometer in spectrometer_data:
    if spectrometer_data[spectrometer]['detectortype'] == 'NIR':
        idx_NIR.append(spectrometer)
    elif spectrometer_data[spectrometer]['detectortype'] == 'VIS':
        idx_VIS.append(spectrometer)
    else:
        print(f"connected PEBBLE {spectrometer_data[spectrometer][serialnumber]} is not VIS or NIR ")
print(idx_NIR, idx_VIS)        

In [ ]:
#use first connected NIR and VIS Pebble to concatenate wavelengthaxis
if idx_NIR != [] and idx_VIS != []:
    wavelengthaxis_total = create_total_wavelengthaxis(idx_NIR[0], idx_VIS[0], int(params['cutoff_NIR_VIS'])) 
    #create total_wavelengthaxis in spectrometer_data
    norm_factor_dark = spectrometer_data[idx_VIS[0]]['darkspec'][0]/spectrometer_data[idx_NIR[0]]['darkspec'][156]
    spectrum1_normalized_dark =np.array(spectrometer_data[idx_NIR[0]]['darkspec']).astype(np.float64) * norm_factor_dark
    spectrum2_unnormalized_dark = np.array(spectrometer_data[idx_VIS[0]]['darkspec']).astype(np.float64)
    spectrum_dark = np.concatenate((spectrum1_normalized_dark[:157], spectrum2_unnormalized_dark[0:]), axis=0)
    #index for new entry
    idx_concatenated_data = len(spectrometer_data)
    spectrometer_data[idx_concatenated_data] = {'wavelengthaxis': wavelengthaxis_total, 'darkspec': spectrum_dark }
    print(f"Concatenated Wavelengthaxis from PEBBLE {spectrometer_data[0]['serialnumber']}"
           + f" and PEBBLE {spectrometer_data[1]['serialnumber']}!")
else:
                                                                         
    print("Missing NIR or VIS Pebble. Could not concatenate wavelengthaxis")

In [ ]:
#create Handles for every spectrometer. Ignore last entry because this is just the concatenated entry.
for spectrometer in range(len(spectrometer_data)):
    if spectrometer > (numberOfSpectrometers -1):
        continue
    else:
        spectrometer_data[spectrometer].update(
            {'handle': create_handle(spectrometer, spectrometer_data[spectrometer]['serialnumber'] )})

In [ ]:
#set exposure Time of every spectrometer
for spectrometer in range(len(spectrometer_data)):
    if spectrometer > (numberOfSpectrometers -1):
        continue
    else:
        IBSEN.set_expTime(spectrometer_data[spectrometer]['handle'], int(params['expTime']), timeUnit= params['timeScale'])
        print(f"Set exposure Time of Spectrometer {spectrometer} to {params['expTime']} {params['timeScale']} seconds ")

In [ ]:
%matplotlib notebook

mplstyle.use(['seaborn', 'fast'])
#create figure
fig = plt.figure()
ax = fig.add_subplot()

ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Counts")

if int(params['dynamic_axis']) == 0:
    ax.set_ylim(0, 65737)
    

#define plot setup given by parameter file 
if int(params['plot_setup']) == 0:
    if int(params['log']) == 0:
        line1, = ax.plot(spectrometer_data[idx_NIR[0]]['wavelengthaxis'], spectrometer_data[idx_NIR[0]]['darkspec'],
                         lw = 0.75, color = 'red', label = f"NIR {spectrometer_data[idx_NIR[0]]['serialnumber']}")
        
    elif int(params['log']) == 1:
        line1, = ax.plot(spectrometer_data[idx_NIR[0]]['wavelengthaxis'], np.log10(spectrometer_data[idx_NIR[0]]['darkspec']),
                         lw = 0.75, color = 'red', label = f"NIR {spectrometer_data[idx_NIR[0]]['serialnumber']}")
        

elif int(params['plot_setup']) == 1:
    if int(params['log']) == 0:
        line1, = ax.plot(spectrometer_data[idx_VIS[0]]['wavelengthaxis'], spectrometer_data[idx_VIS[0]]['darkspec']
                         ,lw = 0.75, color = 'red', label = f"VIS {spectrometer_data[idx_VIS[0]]['serialnumber']}")
        
    elif int(params['log']) == 1:
        line1, = ax.plot(spectrometer_data[idx_VIS[0]]['wavelengthaxis'], np.log10(spectrometer_data[idx_VIS[0]]['darkspec'])
                         ,lw = 0.75, color = 'red',  label = f"VIS {spectrometer_data[idx_VIS[0]]['serialnumber']}")

elif int(params['plot_setup']) == 2: 
    if int(params['log']) == 0:
        line1, = ax.plot(spectrometer_data[idx_concatenated_data]['wavelengthaxis'] 
                         ,spectrometer_data[idx_concatenated_data]['darkspec'], lw = 0.75, color = 'red',
                        label = f"VIS + NIR concatenated and NIR normalized")
        
    elif int(params['log']) == 1:
         line1, = ax.plot(spectrometer_data[idx_concatenated_data]['wavelengthaxis'] 
                         ,np.log10(spectrometer_data[idx_concatenated_data]['darkspec']), lw = 0.75, color = 'red',
                          label = f"VIS + NIR concatenated and NIR normalized")
        
                        
elif int(params['plot_setup']) == 3:
    if int(params['log']) == 0:
        line1, = ax.plot(spectrometer_data[idx_concatenated_data]['wavelengthaxis']
                        ,spectrometer_data[idx_concatenated_data]['darkspec'], lw = 0.75, color = 'red',
                         label = f"VIS + NIR concatenated and NIR normalized")
        line2, = ax.plot(spectrometer_data[idx_NIR[0]]['wavelengthaxis'][156:]
                        ,spectrometer_data[idx_NIR[0]]['darkspec'][156:], lw = 0.75, color = 'gray',
                        label = f"NIR {spectrometer_data[idx_NIR[0]]['serialnumber']} short")
        
    elif int(params['log']) == 1:
        line1, = ax.plot(spectrometer_data[idx_concatenated_data]['wavelengthaxis']
                        ,np.log10(spectrometer_data[idx_concatenated_data]['darkspec']), lw = 0.75, color = 'red',
                        label = f"VIS + NIR concatenated and NIR normalized")
        line2, = ax.plot(spectrometer_data[idx_NIR[0]]['wavelengthaxis'][156:]
                        ,np.log10(spectrometer_data[idx_NIR[0]]['darkspec'][156:]), lw = 0.75, color = 'gray',
                        label = f"NIR {spectrometer_data[idx_NIR[0]]['serialnumber']} short")

elif int(params['plot_setup']) == 4:
    if int(params['log']) == 0:
        line1, = ax.plot(spectrometer_data[idx_concatenated_data]['wavelengthaxis'] 
                         ,spectrometer_data[idx_concatenated_data]['darkspec'], lw = 0.75, color = 'red',
                        label = f"VIS + NIR concatenated and NIR normalized")
        line2, = ax.plot(spectrometer_data[idx_VIS[1]]['wavelengthaxis'], spectrometer_data[idx_VIS[1]]['darkspec']
                         ,lw = 0.75, color = 'blue', label = f"NIR {spectrometer_data[idx_VIS[1]]['serialnumber']}")
        
    elif int(params['log']) == 1:
        line1, = ax.plot(spectrometer_data[idx_concatenated_data]['wavelengthaxis'] 
                    ,np.log10(spectrometer_data[idx_concatenated_data]['darkspec']), lw = 0.75, color = 'red',
                        label = f"VIS + NIR concatenated and NIR normalized")
        line2, = ax.plot(spectrometer_data[idx_VIS[1]]['wavelengthaxis'], np.log10(spectrometer_data[idx_VIS[1]]['darkspec'])
                         ,lw = 0.75, color = 'blue', label = f"NIR {spectrometer_data[idx_VIS[0]]['serialnumber']}")

elif int(params['plot_setup']) == 5:
    if int(params['log']) == 0:
        line1, = ax.plot(spectrometer_data[idx_NIR[0]]['wavelengthaxis'], spectrometer_data[idx_NIR[0]]['darkspec'],
                         lw = 0.75, color = 'red', label = f"NIR {spectrometer_data[idx_NIR[0]]['serialnumber']}")
        line2, = ax.plot(spectrometer_data[idx_VIS[0]]['wavelengthaxis'], spectrometer_data[idx_VIS[0]]['darkspec']
                         ,lw = 0.75, color = 'blue',label = f"NIR {spectrometer_data[idx_VIS[0]]['serialnumber']}")
        line3, = ax.plot(spectrometer_data[idx_VIS[1]]['wavelengthaxis'], spectrometer_data[idx_VIS[1]]['darkspec']
                         ,lw = 0.75, color = 'gray', label = f"NIR {spectrometer_data[idx_VIS[1]]['serialnumber']}")
        
    elif int(params['log']) == 1:
        line1, = ax.plot(spectrometer_data[idx_NIR[0]]['wavelengthaxis'], np.log10(spectrometer_data[idx_NIR[0]]['darkspec']),
                         lw = 0.75, color = 'red', label = f"NIR {spectrometer_data[idx_NIR[0]]['serialnumber']}")
        line2, = ax.plot(spectrometer_data[idx_VIS[0]]['wavelengthaxis'], np.log10(spectrometer_data[idx_VIS[0]]['darkspec'])
                         ,lw = 0.75, color = 'blue', label = f"NIR {spectrometer_data[idx_VIS[0]]['serialnumber']}")
        line3, = ax.plot(spectrometer_data[idx_VIS[1]]['wavelengthaxis'], np.log10(spectrometer_data[idx_VIS[1]]['darkspec'])
                         ,lw = 0.75, color = 'gray',label = f"NIR {spectrometer_data[idx_VIS[1]]['serialnumber']}")
        
plt.legend(loc = "upper right")

#update the parameters to plot with PEBBLE data
def animate(i):
    
    for i in range(0, numberOfSpectrometers):
        spectrometer_data[i].update(
            {'spectrum': IBSEN.acquireSpectra(spectrometer_data[i]['handle'])})
    
    if idx_NIR != []:
        norm_factor = spectrometer_data[idx_VIS[0]]['spectrum'][0]/spectrometer_data[idx_NIR[0]]['spectrum'][156]
        spectrum1_normalized =np.array(spectrometer_data[idx_NIR[0]]['spectrum']).astype(np.float64) * norm_factor
        spectrum2_unnormalized = np.array(spectrometer_data[idx_VIS[0]]['spectrum']).astype(np.float64)
        spectrum = np.concatenate((spectrum1_normalized[:157], spectrum2_unnormalized[0:]), axis=0)                
        spectrometer_data[idx_concatenated_data].update({'spectrum': spectrum})
    
    if int(params['plot_setup']) == 0:
        if int(params['log']) == 0:
            line1.set_ydata(spectrometer_data[idx_NIR[0]]['spectrum'])
            if int(params['dynamic_axis']) == 1:
                ax.set_ylim(0, max(spectrometer_data[idx_NIR[0]]['spectrum']) + 100)
            return line1,
        
        elif int(params['log']) == 1:
            line1.set_ydata(np.log10(spectrometer_data[idx_NIR[0]]['spectrum']))
            if int(params['dynamic_axis']) == 1:
                ax.set_ylim(0, max(np.log10(spectrometer_data[idx_NIR[0]]['spectrum'])) + np.log10(100))
            return line1,
                     
    elif int(params['plot_setup']) == 1:
        if int(params['log']) == 0:
            line1.set_ydata(spectrometer_data[idx_VIS[0]]['spectrum'])
            if int(params['dynamic_axis']) == 1:
                ax.set_ylim(0, max(spectrometer_data[idx_VIS[0]]['spectrum']) + 100)
            return (line1,)
        
        elif int(params['log']) == 1:
            line1.set_ydata(np.log10(spectrometer_data[idx_VIS[0]]['spectrum']))
            if int(params['dynamic_axis']) == 1: 
                ax.set_ylim(0, max(np.log10(spectrometer_data[idx_VIS[0]]['spectrum'])) + np.log10(100))
            return (line1,)
    
    elif int(params['plot_setup']) == 2:
        if int(params['log']) == 0:
            line1.set_ydata(spectrometer_data[idx_concatenated_data]['spectrum'])
            if int(params['dynamic_axis']) == 1:
                ax.set_ylim(0, max(spectrometer_data[idx_concatenated_data]['spectrum']) + 100)
            return (line1,)
        
        elif int(params['log']) == 1:
            line1.set_ydata(np.log10(spectrometer_data[idx_concatenated_data]['spectrum']))
            if int(params['dynamic_axis']) == 1:
                ax.set_ylim(0, max(np.log10(spectrometer_data[idx_concatenated_data]['spectrum'])) + np.log10(100))
            return (line1,)
            
    
    elif int(params['plot_setup']) == 3:
        if int(params['log']) == 0:
            line1.set_ydata(spectrometer_data[idx_concatenated_data]['spectrum'])
            line2.set_ydata(np.array(spectrometer_data[idx_NIR[0]]['spectrum']).astype(np.float64)[156:] * norm_factor)
            if int(params['dynamic_axis']) == 1:
                if max(spectrometer_data[idx_concatenated_data]['spectrum']) > max(np.array(spectrometer_data[idx_NIR[0]]['spectrum']) * norm_factor):
                    ax.set_ylim(0, max(spectrometer_data[idx_concatenated_data]['spectrum']) + 100)
                else:
                    ax.set_ylim(0, max(spectrometer_data[idx_NIR[0]]['spectrum']) * norm_factor + 100)
            return (line1,), (line2,)
        
        elif int(params['log']) == 1:
            line1.set_ydata(np.log10(spectrometer_data[idx_concatenated_data]['spectrum']))
            line2.set_ydata(np.log10(np.array(spectrometer_data[idx_NIR[0]]['spectrum']).astype(np.float64)[156:] * norm_factor))
            if max(spectrometer_data[idx_concatenated_data]['spectrum']) > max(np.array(spectrometer_data[idx_NIR[0]]['spectrum']) * norm_factor):
                ax.set_ylim(0, max(np.log10(spectrometer_data[idx_concatenated_data]['spectrum'])) + np.log10(100))
            else:
                ax.set_ylim(0, max(np.log10(spectrometer_data[idx_NIR[0]]['spectrum'])) * norm_factor + np.log10(100))   
        
    elif int(params['plot_setup']) == 4:
        if int(params['log']) == 0:
            line1.set_ydata(spectrometer_data[idx_concatenated_data]['spectrum'])
            line2.set_ydata(spectrometer_data[idx_VIS[1]]['spectrum'])
            if int(params['dynamic_axis']) == 1:
                if max(spectrometer_data[idx_concatenated_data]['spectrum']) > max(spectrometer_data[idx_VIS[1]]['spectrum']):
                    ax.set_ylim(0, max(spectrometer_data[idx_concatenated_data]['spectrum']) + 100)
                else:
                    ax.set_ylim(0, max(spectrometer_data[idx_VIS[1]]['spectrum']) + 100)
            return (line1,), (line2,)
        
        elif int(params['log']) == 1:
            
            line1.set_ydata(np.log10(spectrometer_data[idx_concatenated_data]['spectrum']))
            line2.set_ydata(np.log10(spectrometer_data[idx_VIS[1]]['spectrum']))
            if int(params['dynamic_axis']) == 1:
                if max(spectrometer_data[idx_concatenated_data]['spectrum']) > max(spectrometer_data[idx_VIS[1]]['spectrum']):
                    ax.set_ylim(0, max(np.log10(spectrometer_data[idx_concatenated_data]['spectrum'])) + np.log10(100))
                else:
                    ax.set_ylim(0, max(np.log10(spectrometer_data[idx_VIS[1]]['spectrum'])) + np.log10(100))
            return (line1,), (line2,)
                     
    elif int(params['plot_setup']) == 5:
        if int(params['log']) == 0:
            line1.set_ydata(spectrometer_data[idx_NIR[0]]['spectrum'])
            line2.set_ydata(spectrometer_data[idx_VIS[0]]['spectrum'])
            line3.set_ydata(spectrometer_data[idx_VIS[1]]['spectrum'])
            
            if int(params['dynamic_axis']) == 1:
                if max(spectrometer_data[idx_VIS[0]]['spectrum']) > max(spectrometer_data[idx_VIS[1]]['spectrum']):
                    if max(spectrometer_data[idx_VIS[0]]['spectrum']) > max(spectrometer_data[idx_NIR[0]]['spectrum']):
                        ax.set_ylim(0, max(spectrometer_data[idx_VIS[0]]['spectrum']) + 100)
                    else:
                        ax.set_ylim(0, max(spectrometer_data[idx_NIR[0]]['spectrum']) + 100)
                else:
                    if max(spectrometer_data[idx_VIS[1]]['spectrum']) > max(spectrometer_data[idx_NIR[0]]['spectrum']):
                        ax.set_ylim(0, max(spectrometer_data[idx_VIS[1]]['spectrum']) + 100)
                    else:
                        ax.set_ylim(0, max(spectrometer_data[idx_NIR[0]]['spectrum']) + 100)
                    
            return (line1,), (line2,), (line3,)  
        
        elif int(params['log']) == 1:        
            line1.set_ydata(np.log10(spectrometer_data[idx_NIR[0]]['spectrum']))
            line2.set_ydata(np.log10(spectrometer_data[idx_VIS[0]]['spectrum']))
            line3.set_ydata(np.log10(spectrometer_data[idx_VIS[1]]['spectrum']))
            
            if int(params['dynamic_axis']) == 1:
                if max(spectrometer_data[idx_VIS[0]]['spectrum']) > max(spectrometer_data[idx_VIS[1]]['spectrum']):
                    if max(spectrometer_data[idx_VIS[0]]['spectrum']) > max(spectrometer_data[idx_NIR[0]]['spectrum']):
                        ax.set_ylim(0, max(np.log10(spectrometer_data[idx_VIS[0]]['spectrum'])) + np.log10(100))
                    else:
                        ax.set_ylim(0, max(np.log10(spectrometer_data[idx_NIR[0]]['spectrum'])) + np.log10(100))
                else:
                    if max(spectrometer_data[idx_VIS[1]]['spectrum']) > max(spectrometer_data[idx_NIR[0]]['spectrum']):
                        ax.set_ylim(0, max(np.log10(spectrometer_data[idx_VIS[1]]['spectrum'])) + np.log10(100))
                    else:
                        ax.set_ylim(0, max(np.log10(spectrometer_data[idx_NIR[0]]['spectrum'])) + np.log10(100))

            return (line1,), (line2,), (line3,)  
    
ani = FuncAnimation(fig, animate, interval = int(params['expTime'])* numberOfSpectrometers + 50, blit = True)

In [ ]:
for spectrometer in range(len(spectrometer_data)):
    if spectrometer > (numberOfSpectrometers -1):
        continue
    else:
        IBSEN.closeDevice(spectrometer_data[spectrometer]['handle'])
        print(spectrometer_data[spectrometer]['handle'])
print("All Spectrometer Closed!")

In [ ]:
def write_spectra_to_file(filename, number):
    
    if 'dark' in filename.lower():
        with open(filename, 'w' ,newline = '') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(range(0,256))
            writer.writerow(spectrometer_data[number]['wavelengthaxis'])
            writer.writerow(spectrometer_data[number]['darkspec'])
        
    else:
        with open(filename, 'w' ,newline = '') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(range(0,256))
            writer.writerow(spectrometer_data[number]['wavelengthaxis'])
            writer.writerow(spectrometer_data[number]['spectrum'])

In [ ]:
#safe spectrum from every PEBBLE
for i in range(len(spectrometer_data)):
    if i > (numberOfSpectrometers -1):
        continue
    else:
        filename = (f"{params['foldername']}" + '/' + f"{params['subfoldername']}" + '/' 
                + f"{datetime.today().strftime('%Y-%m-%d-%H-%M')}_" 
                + f"PEBBLE{spectrometer_data[i]['serialnumber']}_SPECTRUMlive.csv")
        params.update({f'filename_{i}' : filename })
        write_spectra_to_file(filename, i)

In [ ]:
#safe concatenated spectrum raw data
if idx_NIR != [] and idx_VIS != []:
    filename = (f"{params['foldername']}" + '/' + f"{params['subfoldername']}" + '/' 
                + f"{datetime.today().strftime('%Y-%m-%d-%H-%M')}_" 
                + f"PEBBLE{spectrometer_data[idx_NIR[0]]['serialnumber']}_{spectrometer_data[idx_VIS[0]]['serialnumber']}_SPECTRUMlive.csv")
    params.update({f'filename_{numberOfSpectrometers}' : filename })
    with open(filename, 'w' ,newline = '') as csvfile:    
        writer = csv.writer(csvfile)
        writer.writerow(range(0, len(wavelengthaxis_total)))
        writer.writerow(wavelengthaxis_total)
        writer.writerow(spectrometer_data[idx_concatenated_data]['spectrum'])

In [ ]:
#Safe DarkCounts from every PEBLLE
for i in range(len(spectrometer_data)):
    if i > (numberOfSpectrometers -1):
        continue
    else:
        filename = (f"{params['foldername']}" + '/' + f"{params['subfoldername']}" + '/' 
                + f"{datetime.today().strftime('%Y-%m-%d-%H-%M')}_" 
                + f"PEBBLE{spectrometer_data[i]['serialnumber']}_SPECTRUMlivedark.csv")
        params.update({f'filename_{i}' : filename })
        write_spectra_to_file(filename, i)

In [ ]:
#before saving, add number_of_saved_spectra
params.update({'number_of_saved_spectra' : 1})
#save params file
with open(f"{params['foldername']}" + '/' + f"{params['subfoldername']}" + '/' + 'params_for_plot.csv', 'w') as csvfile:
    for paramter, paramter_value in params.items():
        csvfile.writelines(f'{paramter}, {parameter_value} \n')